In [1]:
import pandas as pd
import numpy as np

# Load the Excel file
df = pd.read_csv("plumedata.csv")

headings = [
    "trjNum",
    "vialNum",
    "trialNum",
    "fileindex",
    "trjNumVideo",
    "t",
    "x",
    "y",
    "vx",
    "vy",
    "speed",
    "theta",
    "signal",
    "dtheta",
    "fps",
    "sx",
    "sy",
    "nofflies",
    "starve_day",
    "age_day",
    "room_Temp",
    "room_Hum",
    "onset_num",
    "offset_num",
    "signal_threshold",
    "mu",
    "sigma",
    "signal_mask",
    "vx_smooth",
    "vy_smooth",
    "speed_smooth",
    "x_smooth",
    "y_smooth",
    "theta_smooth",
    "dtheta_smooth",
    "stop_to_go_rate",
    "go_to_stop_rate",
    "switch_rate",
    "stops_on",
    "turns_on"
]


In [4]:
#Need to extrapolate a left and right signal from the dataset, otherwise the sensors in the SNN will have no directionality to go off of. 

# converting angles to rads
theta_rad = np.deg2rad(df.iloc[:, headings.index("theta_smooth")])

# Computing a vector to odor source
dx = df.iloc[:, headings.index("sx")] - df.iloc[:, headings.index("x_smooth")]
dy = df.iloc[:, headings.index("sy")] - df.iloc[:, headings.index("y_smooth")]

# Normalize 
mag = np.sqrt(dx**2 + dy**2) + 1e-6
to_src_x = dx / mag
to_src_y = dy / mag

# Fly heading and perpendicular (left) vector
heading_x = np.cos(theta_rad)
heading_y = np.sin(theta_rad)
heading_perp_x = -heading_y
heading_perp_y = heading_x

# Compute left/right bias (positive = source more to left, negative = right)
bias = to_src_x * heading_perp_x + to_src_y * heading_perp_y

# Normalize bias to range
bias = np.clip(bias, -1, 1)

# Original odor signal
signal = df.iloc[:, headings.index("signal")]

# Compute left and right odors
modulation_strength = 0.05  # This determines how strong the lateral difference is
df["odor_left1"] = signal * (1 + modulation_strength * bias)
df["odor_right1"] = signal * (1 - modulation_strength * bias)
df["odor_left2"] = signal * (1 + modulation_strength * bias)
df["odor_right2"] = signal * (1 - modulation_strength * bias)

df["trjnum"] = df.iloc[:,1]

df_pros = df[["trjnum","odor_left1", "odor_left2", "odor_right1", "odor_right2"]].copy()

In [ ]:
df_pros[["odor_left1", "odor_left2", "odor_right1", "odor_right2"]] = df[["odor_left1", "odor_left2", "odor_right1", "odor_right2"]].clip(lower=0)

KeyError: ('odor_left1', 'odor_left2', 'odor_right1', 'odor_right2')

In [5]:
# Add a column showing animal activity, i.e. forwards, turn_left, turn_right, stop

# Determine turning direction
dtheta = df.iloc[:, headings.index("dtheta_smooth")]
turns_on = df.iloc[:, headings.index("turns_on")]
stops_on = df.iloc[:, headings.index("stops_on")]

# Default behavior is forward
labels = np.zeros(len(df), dtype=int)

# Label STOP as 3
labels[stops_on >= 1] = 3

# Label TURN_LEFT (positive dtheta) as 1
labels[(turns_on >= 1) & (dtheta > 0)] = 1

# Label TURN_RIGHT (negative dtheta) as 2
labels[(turns_on >= 1) & (dtheta < 0)] = 2

df_pros["label"] = labels


df_pros.to_csv("processed_fly_data.csv", index=False)
